# Synthetic Validation of Robust PCA using ADMM

In this notebook, we validate our implementation of **Robust Principal Component Analysis (RPCA)**
solved using the **Alternating Direction Method of Multipliers (ADMM)** using synthetic data.

The goal is to decompose a data matrix $M$ into:
$$
M = L + S,
$$
where:
- $L$ is **low-rank** and captures the underlying structure,
- $S$ is **sparse** and captures outliers or corruptions.

Since the synthetic data are generated with known ground-truth components, this experiment allows us to verify:
- convergence of the ADMM algorithm,
- satisfaction of the constraint $M \approx L + S$,
- correct recovery of the low-rank and sparse structures.


In [2]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display

## Experiment results

The synthetic RPCA experiment was executed using the script:

`experiments/synthetic_rpca.py`

All generated outputs (heatmaps, convergence plot and log file) are stored in:

`experiments/results/synthetic/`

This notebook loads and analyzes those results without re-running the solver.


In [ ]:
results_dir = os.path.join("..", "experiments", "results", "synthetic")
results_dir
